In [1]:
from sklearn.neural_network import MLPClassifier
from sklearn import datasets, preprocessing
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

In [2]:
iris = datasets.load_iris()

In [3]:
X = preprocessing.normalize(iris.data[:, :2],axis=1)  # we only take the first two features.
y = iris.target

In [4]:
mask = y != 2
X = X[mask]
y = y[mask]

In [5]:
y == 0

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False, False], dtype=bool)

In [6]:
y1 = np.matrix([np.array(y == 0),np.array(y == 1)]).transpose()

In [7]:
y1.shape, X.shape

((100, 2), (100, 2))

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y1, test_size=0.33, random_state=42)

In [9]:
clf = MLPClassifier(solver='lbfgs', max_iter=1, warm_start=True,
                    hidden_layer_sizes=(3), random_state=1)

In [10]:
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.42424242424242425

In [11]:
clf.coefs_[0].shape, clf.coefs_[1].shape

((2, 3), (3, 2))

In [12]:
clf.coefs_[0]

array([[-0.18179391,  0.48270209, -1.09518389],
       [-0.43306343, -0.77391155, -0.89313271]])

In [13]:
clf.coefs_[1]

array([[ 0.08504238, -0.17703423],
       [ 0.40579165, -0.64750639],
       [ 0.82840575, -1.03543185]])

In [14]:
clf.intercepts_[0].shape, clf.intercepts_[1].shape

((3,), (2,))

In [15]:
clf.intercepts_

[array([-0.68736944, -0.33835949, -0.22617113]),
 array([-0.47017521,  0.35571616])]

In [88]:
%%writefile led_map.py

def unwrap_to_pairs(l):
    o = []
    if type(l[0]) == list:
        for i in l:
            x = unwrap_to_pairs(i)
            if type(x[0]) != list:
                o += [x]
            else:
                o += x
        return o
    else:
        print(l)
        return l

input_layer_bias = [
                    [8,9],
                    [3,4]
                    ]

input_layer_weights = [
                        [[5,11],[6,20],[7,28]],
                        [[0,10],[1,18],[2,27]]
                        
                      ]

hidden_layer_bias = [
                        [13,14],
                        [21,22],
                        [29,30]
                    ]

hidden_layer_weights = [
                            [[16,41],[15,36]],
                            [[23,40],[25,35]],
                            [[31,39],[32,34]]
                       ]

output_layer_bias = [
                        [42,43],
                        [37,38]
                    ]

weight_index =  (unwrap_to_pairs(input_layer_weights) + 
                 unwrap_to_pairs(hidden_layer_weights)
                )

bias_index =    (input_layer_bias +
                 hidden_layer_bias +
                 output_layer_bias)

def map_weight_to_led(coefs, led_map):
    leds = []
    for i,v in enumerate(led_map):
        for i2,v2 in enumerate(v):
            print(i,i2,v2,coefs[i][i2])
            for led in v2:
                leds.append({"led_num":led, "val":coefs[i][i2]})
    return leds

def map_bias_to_led(bias, led_map):
    leds = []
    for i,v in enumerate(led_map):
        print(i,v,bias[i])
        for led in v:
            leds.append({"led_num":led, "val":bias[i]})
    return leds

def make_led_coef_vals(clf):
    leds = []
    leds += map_weight_to_led(clf.coefs_[0], input_layer_weights)
    leds += map_weight_to_led(clf.coefs_[1], hidden_layer_weights)
    leds += map_bias_to_led(clf.intercepts_[0], hidden_layer_bias)
    leds += map_bias_to_led(clf.intercepts_[0], output_layer_bias)
    return leds

Overwriting led_map.py


In [89]:
weight_index

[[5, 12],
 [6, 20],
 [7, 26],
 [0, 10],
 [1, 18],
 [2, 26],
 [16, 41],
 [15, 36],
 [23, 40],
 [25, 35],
 [31, 39],
 [32, 34]]

In [83]:
bias_index

[[8, 9], [3, 4], [13, 14], [21, 22], [29, 30], [42, 43], [37, 38]]

In [109]:
map_weight_to_led(clf.coefs_[1], hidden_layer_weights)

0 0 [16, 41] 0.0850423770945
0 1 [15, 36] -0.177034229022
1 0 [23, 40] 0.405791651524
1 1 [25, 35] -0.647506387982
2 0 [33, 39] 0.828405749149
2 1 [32, 34] -1.03543184533


[{'led_num': 16, 'val': 0.085042377094536728},
 {'led_num': 41, 'val': 0.085042377094536728},
 {'led_num': 15, 'val': -0.17703422902163454},
 {'led_num': 36, 'val': -0.17703422902163454},
 {'led_num': 23, 'val': 0.40579165152433216},
 {'led_num': 40, 'val': 0.40579165152433216},
 {'led_num': 25, 'val': -0.6475063879820655},
 {'led_num': 35, 'val': -0.6475063879820655},
 {'led_num': 33, 'val': 0.82840574914925591},
 {'led_num': 39, 'val': 0.82840574914925591},
 {'led_num': 32, 'val': -1.0354318453310034},
 {'led_num': 34, 'val': -1.0354318453310034}]

In [110]:
map_bias_to_led(clf.intercepts_[0],hidden_layer_bias)

0 [13, 14] -0.687369437661
1 [21, 22] -0.338359494253
2 [29, 30] -0.226171132128


[{'led_num': 13, 'val': -0.68736943766140923},
 {'led_num': 14, 'val': -0.68736943766140923},
 {'led_num': 21, 'val': -0.33835949425288125},
 {'led_num': 22, 'val': -0.33835949425288125},
 {'led_num': 29, 'val': -0.22617113212838169},
 {'led_num': 30, 'val': -0.22617113212838169}]

In [113]:
f = make_led_coef_vals(clf)
sorted(f, key=lambda k: k['led_num']) 

0 0 [0, 11] -0.181793913059
0 1 [1, 18] 0.48270209062
0 2 [2, 26] -1.09518389253
1 0 [5, 12] -0.433063428162
1 1 [6, 20] -0.773911548997
1 2 [7, 27] -0.893132713009
0 0 [16, 41] 0.0850423770945
0 1 [15, 36] -0.177034229022
1 0 [23, 40] 0.405791651524
1 1 [25, 35] -0.647506387982
2 0 [33, 39] 0.828405749149
2 1 [32, 34] -1.03543184533
0 [13, 14] -0.687369437661
1 [21, 22] -0.338359494253
2 [29, 30] -0.226171132128
0 [42, 43] -0.687369437661
1 [37, 38] -0.338359494253


[{'led_num': 0, 'val': -0.18179391305878864},
 {'led_num': 1, 'val': 0.48270209062025637},
 {'led_num': 2, 'val': -1.0951838925254538},
 {'led_num': 5, 'val': -0.43306342816209792},
 {'led_num': 6, 'val': -0.77391154899731696},
 {'led_num': 7, 'val': -0.89313271300883934},
 {'led_num': 11, 'val': -0.18179391305878864},
 {'led_num': 12, 'val': -0.43306342816209792},
 {'led_num': 13, 'val': -0.68736943766140923},
 {'led_num': 14, 'val': -0.68736943766140923},
 {'led_num': 15, 'val': -0.17703422902163454},
 {'led_num': 16, 'val': 0.085042377094536728},
 {'led_num': 18, 'val': 0.48270209062025637},
 {'led_num': 20, 'val': -0.77391154899731696},
 {'led_num': 21, 'val': -0.33835949425288125},
 {'led_num': 22, 'val': -0.33835949425288125},
 {'led_num': 23, 'val': 0.40579165152433216},
 {'led_num': 25, 'val': -0.6475063879820655},
 {'led_num': 26, 'val': -1.0951838925254538},
 {'led_num': 27, 'val': -0.89313271300883934},
 {'led_num': 29, 'val': -0.22617113212838169},
 {'led_num': 30, 'val': -

In [28]:
a = [*input_layer_weights]

In [78]:
unwrap(input_layer_weights)

[0, 11]
[1, 18]
[2, 26]
[5, 12]
[6, 20]
[7, 27]


[[0, 11], [1, 18], [2, 26], [5, 12], [6, 20], [7, 27]]

In [75]:
type(input_layer_weights) == list

True

In [76]:
unwrap2(input_layer_weights)

[0, 11]
[1, 18]
[2, 26]
[5, 12]
[6, 20]
[7, 27]


[[0, 11], [1, 18], [2, 26], [5, 12], [6, 20], [7, 27]]

In [71]:
a = [9,9] 
a += [3,4]
a

[9, 9, 3, 4]